# Import variables

In [2]:
!pip install hyperopt

     |████████████████████████████████| 965 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 1.6 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=c50f7ebbefb93f8cff5fe6cdb1ec7bcc9151307d8695e43fb037e4fbb7076cf8
  Stored in directory: /home/admin/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future


In [3]:
from hyperopt import tpe
from hyperopt import STATUS_OK
from hyperopt import Trials
from hyperopt import hp
from hyperopt import fmin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import pandas
import mlflow

# Load Data

In [6]:
pandas_df = pandas.read_csv("training_data.csv")
X=pandas_df.iloc[:,:-1]
y=pandas_df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=4284, stratify=y)

# Define objective function

In [7]:
N_FOLDS = 4
MAX_EVALS = 10

def objective(params, n_folds = N_FOLDS):
    """Objective function for Logistic Regression Hyperparameter Tuning"""

    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evaluate based on ROC AUC
    mlflow.sklearn.autolog()
    with mlflow.start_run(nested=True):
        clf = LogisticRegression(**params,random_state=0,verbose =0)
        scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_macro')

        # Extract the best score
        best_score = max(scores)

        # Loss must be minimized
        loss = 1 - best_score

        # Dictionary with information for evaluation
        return {'loss': loss, 'params': params, 'status': STATUS_OK}

# Define parameter space

In [8]:

space = {
    'warm_start' : hp.choice('warm_start', [True, False]),
    'fit_intercept' : hp.choice('fit_intercept', [True, False]),
    'tol' : hp.uniform('tol', 0.00001, 0.0001),
    'C' : hp.uniform('C', 0.05, 3),
    'solver' : hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']),
    'max_iter' : hp.choice('max_iter', range(5,1000))
}

# Create experiment

In [12]:
mlflow.set_experiment("Hyperopt_Optimization")

INFO: 'Hyperopt_Optimization' does not exist. Creating a new experiment


# Define Optimization Trials

In [13]:
# Algorithm
tpe_algorithm = tpe.suggest

# Trials object to track progress
bayes_trials = Trials()




with mlflow.start_run():
    best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = MAX_EVALS, trials = bayes_trials)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2021/02/17 17:46:00 INFO mlflow.utils.autologging_utils: sklearn autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow to the MLflow run with ID 'd124e0956f2a41a98a108d77439be27d'
2021/02/17 17:46:04 WARNING mlflow.utils.autologging_utils: MLflow issued a warning during sklearn autologging: "/opt/conda/lib/python3.7/site-packages/mlflow/models/signature.py:123: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing V

 10%|█         | 1/10 [00:06<01:01,  6.84s/trial, best loss: 0.48659700136301676]

2021/02/17 17:46:07 INFO mlflow.utils.autologging_utils: sklearn autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow to the MLflow run with ID '09da40c242ad4e1ca0f8ea908bf94621'
2021/02/17 17:46:08 WARNING mlflow.utils.autologging_utils: MLflow issued a warning during sklearn autologging: "/opt/conda/lib/python3.7/site-packages/mlflow/models/signature.py:123: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing V

 20%|██        | 2/10 [00:10<00:38,  4.78s/trial, best loss: 0.4753470735734817] 

...

 80%|████████  | 8/10 [00:30<00:06,  3.49s/trial, best loss: 0.4753470735734817]

2021/02/17 17:46:31 INFO mlflow.utils.autologging_utils: sklearn autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow to the MLflow run with ID '8d5c72ce6f564d46bcd631d13d9009ac'
2021/02/17 17:46:31 WARNING mlflow.utils.autologging_utils: MLflow issued a warning during sklearn autologging: "/opt/conda/lib/python3.7/site-packages/mlflow/models/signature.py:123: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing V

 90%|█████████ | 9/10 [00:33<00:03,  3.41s/trial, best loss: 0.4753470735734817]

2021/02/17 17:46:34 INFO mlflow.utils.autologging_utils: sklearn autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow to the MLflow run with ID 'bc880203cd934e44b68dbbc0d0b4701c'
2021/02/17 17:46:35 WARNING mlflow.utils.autologging_utils: MLflow issued a warning during sklearn autologging: "/opt/conda/lib/python3.7/site-packages/mlflow/models/signature.py:123: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing V

100%|██████████| 10/10 [00:37<00:00,  3.74s/trial, best loss: 0.4753470735734817]


In [14]:
best

{'C': 1.852303246830638,
 'fit_intercept': 0,
 'max_iter': 944,
 'solver': 1,
 'tol': 8.551889162947398e-05,
 'warm_start': 0}